# 2243

I think I will choose reference ID 2243 (AB1IP_HUMAN, also known as RIAM) as an example where the alignment is particularly problematic

In [2]:
import local_seqtools.general_utils as tools
import pandas as pd
import numpy as np
import local_conservation_analysis_pipeline.group_conservation_objects as group_tools
import local_seqtools.cli_wrappers as cli
import json
import copy
import os
import re
import sys
from pathlib import Path
from Bio import AlignIO, Seq, SeqIO, Align

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
def filepath_converter(filepath):
    return filepath.replace('/home/jch/Documents/07-pt1_pt2_combined', '../../..')

In [4]:
aln_file = "./ena_motif_alignments/2243_AB1IP_HUMAN_9606_0:00294e/9606_0_00294e_Vertebrata_1587055at7742_clustered_ldos_aln.fasta"
table = pd.read_csv("./enah_table.csv")
og = group_tools.ConserGene(table.loc[table['reference_index'] == 2243]['json_file'].values[0], filepath_converter=filepath_converter)
lvl = og.get_level_obj("Vertebrata", filepath_converter=filepath_converter)
x = tools.sort_aln_by_pid2ref(lvl.aln, lvl.query_aln_sequence)
subsampled_aln_list = tools.subsample_seqrec_list_target_num(x, target_num=20)


In [5]:
mafft_command, mafft_output = cli.mafft_align_wrapper(tools.strip_dashes_from_sequences(subsampled_aln_list))

In [9]:
query_aln_str = str(mafft_output[og.query_gene_id].seq)
_, aln_index = tools.reindex_alignment_str(query_aln_str)
idr_aln_start = aln_index[og.idr_start]
idr_aln_end = aln_index[og.idr_end]
new_aln_list = list(mafft_output.values())
with open(f'../../../../11-pairk/pairk/examples/example_alignment_{og.query_gene_id.replace(":", "_")}-idraln-{idr_aln_start}_to_{idr_aln_end}-idr-{og.idr_start}_to_{og.idr_end}.fasta', 'w') as f:
    SeqIO.write(new_aln_list, f, 'fasta')